In [ ]:
import pathlib

import numpy as np
from matplotlib import pyplot as plt

plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 14

In [ ]:
def read_mqt_predictor_file(num_qubits: int, device: str) -> list[float]:
    best_res = [1.0]
    for file in pathlib.Path("results/").glob(f"mqt_predictor_{num_qubits}_qubits*"):
        found_device = False
        with pathlib.Path(file).open() as f:
            lines = f.readlines()
            for line in lines:
                if line.startswith("Device:") and line.split(":")[1].strip() == device:
                    found_device = True
                if line.startswith("evaluation_data:") and found_device:
                    eval_data = eval(line.split(":")[1])
                    if min(eval_data) < min(best_res):
                        best_res = eval_data
    return best_res


def read_baseline_date(path: str) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    data_baseline = np.loadtxt(path)
    sorted_data = sorted(data_baseline, key=lambda x: min(x))
    best_run = sorted_data[0]
    worst_run = sorted_data[-1]
    median_run = sorted_data[len(sorted_data) // 2]
    return best_run, worst_run, median_run


def generate_eval_plot(
    best_run_O3: np.ndarray,
    worst_run_O3: np.ndarray,
    median_run_O3: np.ndarray,
    median_run_O0,
    mqt_predictor: list[float],
    device: str,
    num_qubits: int,
):
    plt.plot(median_run_O0, label="O0 median", color="black")
    plt.plot(worst_run_O3, label="O3 worst", color="red")
    plt.plot(median_run_O3, label="O3 median", color="orange")
    plt.plot(best_run_O3, label="O3 best", color="green")
    plt.plot(mqt_predictor, label="MQT Predictor")
    plt.title(f"Evaluation for {num_qubits} qubits on {device}")
    plt.yscale("log")
    # plt.ylim(0.12,0.2)
    plt.ylabel("KL Divergence")
    plt.xlabel("Episodes")
    plt.legend()
    plt.savefig(f"results/{num_qubits}_qubits_{device}.pdf", bbox_inches="tight")

# 4 Qubits

In [ ]:
best_run_O3, worst_run_O3, median_run_O3 = read_baseline_date("results/O3/all_eval_data_fake_quito_4_qubits.txt")
_, _, median_run_O0 = read_baseline_date("results/O0/all_eval_data_fake_quito_4_qubits.txt")
mqt_predictor_4_qubits = read_mqt_predictor_file(num_qubits=4, device="ibm_quito")

generate_eval_plot(best_run_O3, worst_run_O3, median_run_O3, median_run_O0, mqt_predictor_4_qubits, "FakeQuito", 4)
print("Diff: ", 1 - min(mqt_predictor_4_qubits) / min(best_run_O3))

# 6 Qubits

In [ ]:
best_run_O3, worst_run_O3, median_run_O3 = read_baseline_date("results/O3/all_eval_data_fake_nairobi_6_qubits.txt")
_, _, median_run_O0 = read_baseline_date("results/O0/all_eval_data_fake_nairobi_6_qubits.txt")
mqt_predictor_6_qubits = read_mqt_predictor_file(num_qubits=6, device="ibm_nairobi")

generate_eval_plot(best_run_O3, worst_run_O3, median_run_O3, median_run_O0, mqt_predictor_6_qubits, "FakeNairobi", 6)
print("Diff: ", 1 - min(mqt_predictor_6_qubits) / min(best_run_O3))

# 8 Qubits Guadalupe

In [ ]:
best_run_O3, worst_run_O3, median_run_O3 = read_baseline_date("results/O3/all_eval_data_fake_guadalupe_8_qubits.txt")
_, _, median_run_O0 = read_baseline_date("results/O0/all_eval_data_fake_guadalupe_8_qubits.txt")
mqt_predictor_8_qubits = read_mqt_predictor_file(num_qubits=8, device="ibm_guadalupe")

generate_eval_plot(best_run_O3, worst_run_O3, median_run_O3, median_run_O0, mqt_predictor_8_qubits, "FakeGuadalupe", 8)
print("Diff: ", 1 - min(mqt_predictor_8_qubits) / min(best_run_O3))

# 8 Qubits Toronto

In [ ]:
best_run_O3, worst_run_O3, median_run_O3 = read_baseline_date("results/O3/all_eval_data_fake_toronto_8_qubits.txt")
_, _, median_run_O0 = read_baseline_date("results/O0/all_eval_data_fake_toronto_8_qubits.txt")
mqt_predictor_8_qubits = read_mqt_predictor_file(num_qubits=8, device="ibm_toronto")

print(1 - min(best_run_O3))
generate_eval_plot(best_run_O3, worst_run_O3, median_run_O3, median_run_O0, mqt_predictor_8_qubits, "FakeToronto", 8)
print("Diff: ", 1 - min(mqt_predictor_8_qubits) / min(best_run_O3))

# 8 Qubits Montreal

In [ ]:
best_run_O3, worst_run_O3, median_run_O3 = read_baseline_date("results/O3/all_eval_data_fake_montreal_8_qubits.txt")
_, _, median_run_O0 = read_baseline_date("results/O0/all_eval_data_fake_montreal_8_qubits.txt")
mqt_predictor_8_qubits = read_mqt_predictor_file(num_qubits=8, device="ibm_montreal")

print(1 - min(best_run_O3))
generate_eval_plot(best_run_O3, worst_run_O3, median_run_O3, median_run_O0, mqt_predictor_8_qubits, "FakeMontreal", 8)
print("Diff: ", 1 - min(mqt_predictor_8_qubits) / min(best_run_O3))